In [1]:
using System.Linq;
class TrapezoidalRule {
    public static double Solve(Func<double, double> f, double a, double b, double dx) {
        if (dx < 1E-7 || b < a || !Double.IsFinite(a + b + dx)) throw new ArgumentException("Неверные данные");
        if (Math.Abs(b-a) < 1E-7) return 0;
        double sum = 0.5 * (a+b) + Enumerable.Range(1, Convert.ToInt32((b-a)/dx) - 1).Sum(i => f(a+i*dx));
        return sum * dx;
    }
}

In [2]:
Func<double, double> f = (double x) => -x*x + 9;

var answ = TrapezoidalRule.Solve(f, -3, 3, 0.1);
answ

35.99

In [3]:
#r "nuget: Newtonsoft.Json, 13.0.3"
using Newtonsoft.Json.Linq;
using System.IO;
public class Student{
    public string name { get; set; }
    public string group { get; set; }
    public string discipline { get; set; }
    public int mark { get; set; }
}

Installed Packages Newtonsoft.Json, 13.0.3

In [4]:
public void Task2 (string inputPath, string outputPath) {
    JObject json = JObject.Parse(File.ReadAllText(inputPath));
    IList<Student> students = json["data"].Select(p => new Student
    {
        name = (string)p["name"],
        group = (string)p["group"],
        discipline = (string)p["discipline"],
        mark = (int)p["mark"]
    }).ToList();
    string taskName = (string)json["taskName"];
    switch (taskName){
        case "GetStudentsWithHighestGPA":
            var nameGroups = students.GroupBy(x => x.name).Select(p => new {Cadet = p.Key, GPA = p.Average(g => g.mark)});
            var bestGPA = nameGroups.Where(x => x.GPA == nameGroups.Max(p => p.GPA));
            JObject answer1 = new JObject
            {
                new JProperty("Response", 
                    new JArray 
                    (
                        bestGPA.Select(x => new JObject 
                        {
                            new JProperty ("Cadet", x.Cadet),
                            new JProperty ("GPA", x.GPA),
                        })
                    )
                )
            };
            File.WriteAllText(outputPath, answer1.ToString());
            break;
        

        case "CalculateGPAByDiscipline":
            var disciplineGroups = students.GroupBy(x => x.discipline).Select(p => new {Discipline = p.Key, GPA = p.Average(g => g.mark)});
            JObject answer2 = new JObject
            {
                new JProperty("Response", 
                    new JArray 
                    (
                        disciplineGroups.Select(x => new JObject 
                        {
                            new JProperty (x.Discipline, x.GPA),
                        })
                    )
                )
            };
            File.WriteAllText(outputPath, answer2.ToString());
            break;

        
        case "GetBestGroupsByDiscipline":
            var disciplineAndGroups = students.GroupBy(x => new {x.discipline, x.group}).Select(p => new {Discipline = p.Key.discipline, Group = p.Key.group, GPA = p.Average(g => g.mark)});
            var bestGroups = disciplineAndGroups.Where(x => x.GPA == disciplineAndGroups.Max(p => p.GPA));
            JObject answer3 = new JObject
            {
                new JProperty("Response", 
                    new JArray 
                    (
                        bestGroups.Select(x => new JObject 
                        {
                            new JProperty ("Discipline", x.Discipline),
                            new JProperty ("Group", x.Group),
                            new JProperty ("GPA", x.GPA)
                        })
                    )
                )
            };
            File.WriteAllText(outputPath, answer3.ToString());
            break;
    }
}

In [5]:
Task2("GetStudentsWithHighestGPA.txt", "output.txt");

In [6]:
Task2("CalculateGPAByDiscipline.txt", "output.txt");

In [7]:
Task2("GetBestGroupsByDiscipline.txt", "output.txt");